In [1]:
import json
import http.client

import http
import time
import random


from urllib import request

In [17]:
API_KEY = 'API KEY'

In [ ]:
def merge_dicts(*dict_args):
    """
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    """
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

In [ ]:
for year in range(1960, 1970):

    print('%%%%%%%%%%%%%%%%%%%%%%')
    print(year)
    print('%%%%%%%%%%%%%%%%%%%%%%')
    movies = list()
    page = 1
    last_page = 10 # temp value

    conn = http.client.HTTPSConnection("api.themoviedb.org")

    payload = "{}"

    while(page <= last_page):
        uri = "/3/discover/movie?page={}&primary_release_year={}&include_adult=true&sort_by=popularity.desc&api_key={}".format(page, year, API_KEY)
        conn.request("GET", uri, payload)
        res = conn.getresponse()
        data = json.loads(res.read().decode('utf-8'))
        movies.extend(data['results'])
        last_page = data['total_pages']
        page = data['page'] + 1
        if(page%100 == 0):
            print(page)
        time.sleep(0.4)

    with open('themoviedb-{}.json'.format(year), 'w') as fp:
        json.dump(movies, fp)


    index = 0
    total_movies = len(movies)

    conn = http.client.HTTPSConnection("api.themoviedb.org")

    payload = "{}"

    while(index < total_movies):
        try:
            movie = movies[index]
            uri = "/3/movie/{}?api_key={}".format(movie['id'], API_KEY)
            conn.request("GET", uri, payload)
            res = conn.getresponse()
            data = json.loads(res.read().decode('utf-8'))
            movies[index] = merge_dicts(movie, data)
            index = index + 1
            if(index%100 == 0):
                print(index)
            time.sleep(0.4)
        except:
            print('what')
            with open('themoviedb-{}.json'.format(year), 'w') as fp:
                json.dump(movies, fp)
            break

    with open('themoviedb-{}-details.json'.format(year), 'w') as fp:
        json.dump(movies, fp)

In [ ]:
#!/bin/bash
#for i in {1970..2007}
#do
#   scp -r root@162.243.209.144:/themoviedb-$i-details.json .
#done

In [ ]:
data = []
for year in range(1970, 1980):
    with open('../themoviedb-{}-details.json'.format(year), encoding='utf-8') as data_file:
        data.extend(json.loads(data_file.read()))

with open('themoviedb-1970-1979-details', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
data = []
for year in range(1980, 1990):
    with open('../themoviedb-{}-details.json'.format(year), encoding='utf-8') as data_file:
        data.extend(json.loads(data_file.read()))
        
with open('themoviedb-1980-1989-details', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
data = []
for year in range(1990, 2000):
    with open('../themoviedb-{}-details.json'.format(year), encoding='utf-8') as data_file:
        data.extend(json.loads(data_file.read()))
        
with open('themoviedb-1990-1999-details', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
data = []
for year in range(2000, 2017):
    with open('../themoviedb-{}-details.json'.format(year), encoding='utf-8') as data_file:
        data.extend(json.loads(data_file.read()))
        
with open('themoviedb-2000-2016-details', 'w') as outfile:
    json.dump(data, outfile)

In [3]:
all_movies = []

for year in range(1970, 2017):
    with open('../themoviedb-{}-details.json'.format(year), encoding='utf-8') as data_file:
        all_movies.extend(json.loads(data_file.read()))
        
with open('themoviedb-all-movies-details.json', 'w') as outfile:
    json.dump(all_movies, outfile)

## number of movies

In [4]:
len(all_movies)

226828

In [5]:
movies_with_genres = [movie for movie in all_movies if len(movie['genre_ids']) > 0]

## number of movies with a least one genre

In [6]:
len(movies_with_genres)

145900

In [7]:
with open('themoviedb-all-movies-with_genres-details.json', 'w') as outfile:
    json.dump(movies_with_genres, outfile)

In [8]:
english_movies_with_genres = list()

for movie in movies_with_genres:
    if not "spoken_languages" in movie:
        continue
        
    spoken_languages = movie["spoken_languages"]
    
    eng = [lang for lang in spoken_languages if lang['iso_639_1'] == 'en' ]
    
    if len(eng) > 0:
        english_movies_with_genres.append(movie)

## number of movies, with at least 1 genre and has english

In [9]:
len(english_movies_with_genres)

54410

In [10]:
with open('themoviedb-all-movies-with_genres-details-english', 'w') as outfile:
    json.dump(english_movies_with_genres, outfile)

In [11]:
english_movies_with_genres_posters = [movie for movie in english_movies_with_genres if movie['poster_path']]

## number of movies, with at least 1 genre, has english and a poster 

In [12]:
len(english_movies_with_genres_posters)

51512

In [13]:
indices = random.sample(range(len(english_movies_with_genres_posters)), 5000)
random_5k_movies = [english_movies_with_genres_posters[i] for i in indices]

with open('themoviedb-english_movies_with_genres-with_genres-details-english.json', 'w') as outfile:
    json.dump(random_5k_movies, outfile)

In [ ]:
random_5k_movies = []
with open('themoviedb-english_movies_with_genres-with_genres-details-english.json', encoding='utf-8') as data_file:
    random_5k_movies.extend(json.loads(data_file.read()))

random_5k_movies_with_cast_crew = list()

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"

index = 0 

for movie in random_5k_movies:
        uri = "/3/movie/{}/credits?api_key={}".format(movie['id'], API_KEY)
        conn.request("GET", uri, payload)
        res = conn.getresponse()
        data = json.loads(res.read().decode('utf-8'))
        time.sleep(0.5)
        
        movie['cast'] = data.get("cast", [])
        
        movie['crew'] = data.get("crew", [])
        
        random_5k_movies_with_cast_crew.append(movie)
            
with open('themoviedb-5k-sample-with_genres-details-english.json', 'w') as outfile:
    json.dump(random_5k_movies_with_cast_crew, outfile)

len(random_5k_movies_with_cast_crew)   

In [ ]:
with open('themoviedb-5k-sample-with_genres-details-english.json', 'w') as outfile:
    json.dump(random_5k_movies_with_cast_crew, outfile)

In [ ]:
random_5k_movies_with_cast_crew = list()
with open('themoviedb-5k-sample-with_genres-details-english.json', encoding='utf-8') as data_file:
    random_5k_movies_with_cast_crew.extend(json.loads(data_file.read()))

In [ ]:
for movies in random_5k_movies_with_cast_crew:
    poster_path = movies['poster_path']
    request.urlretrieve("https://image.tmdb.org/t/p/w500/{}".format(poster_path)
                        , "./data/{}".format(poster_path))
    time.sleep(0.1)

In [ ]:
len([movies for movies in random_5k_movies_with_cast_crew if movies['poster_path']])